# Data and Uncertainty

- In probability, underlying processes are represented by a __model__ (often some distribution)   
    e.g., flipping a coin, rolling a dice, etc.
    
- In reality, even for very simple processes, underlying process is not easily described   
    e.g., what is the true $p$ of a coin? pmf of a dice?,
    
- Many real-world processes are much more complicated; however, we still try  
    > _Now it would be very remarkable if any system existing in the real world could be 
    exactly represented by any simple model. However, cunningly chosen parsimonious models 
    often do provide remarkably useful approximations. 
    [... example of modeling gas in physics ...]  
    For such a model there is no need to ask the question "Is the model true?". 
    If "truth" is to be the "whole truth" the answer must be "No". 
    The only question of interest is "Is the model illuminating and useful?"._  
    George Box

- If you build a model with random variables having known distributions, we can simulate the outcomes

- If the model is accurate to some degree, we can say a lot about the outcome: i.e., the model is useful

- However, often building models from ground up is practically impossible
    e.g., where do you begin to build a model for NCAA tournament? human behavior?

- Often we are interested in inference and/or prediction

    - __Inference__: making conclusions based on data  
        e.g., parameters such as winning probability for each team

    - __Prediction__: making guesses about the next outcome  
        e.g., who will win?
        
- Monte Carlo simulation is a good tool to illustrate the relationship between the generating process and data

## Monte Carlo simulation

First, recall the definition of a random sample:

> __Random sample__: Let $X_1,X_2,\dots,X_n$ be a set of $n$ independent random variables, 
all having the _same pdf_, $f_X$. Then $X_1,X_2,\dots,X_n$ are said to be a _random sample of size n_.

- Random sample: $X_1,X_2,\dots,X_n$

- Random measurements: $x_1,x_2,\dots,x_n$

While random sample is made up of random variables,
__iid observations__ (independent identically distributed)
$x_1, x_2,\dots x_n$ are $n$-measurements (numbers) coming from $X_1,X_2,\dots,X_n$

- In order to capture randomness of _random sample_ with _iid observations_, 
    we can generate many sets of random measurements   
    $$
    x_1^{(1)},x_2^{(1)},\dots,x_n^{(1)}\\
    x_1^{(2)},x_2^{(2)},\dots,x_n^{(2)}\\
    \vdots\\
    x_1^{(S)},x_2^{(S)},\dots,x_n^{(S)}
    $$

- As an example, let's simulate convergence of the sample mean to the true mean

### Consistency of the sample mean: dice example

Suppose that $X_1,X_2,\dots,X_n$ is a random sample of size $n$ from a discrete pdf $p_X(k;\mu)$, where $E(X)=\mu$ and $\text{Var}(X) = \sigma^2 < \infty$. Let $\hat\mu_n = \frac{1}{n}\sum_i^n X_i$. Is $\hat\mu_n$ a consistent estimator for $\mu$?

Since we are simulating dice throw with a pmf, the _model_ is perfect. Hence, we know the true value of $E(X)=\mu=3.5$ and $\text{Var}(X) = \sigma^2 = 2.917$. Also, sampling distribution says $E(\hat\mu_n)=\mu$ and $$\text{Var}(\hat\mu_n) = \text{Var}\left(\frac{1}{n}\sum_{i=1}^n X_i \right) = \frac{1}{n^2} \sum_{i=1}^n \text{Var}(X_i)= \frac{\sigma^2}{n}$$. We will verify this empirically.

First, generate many random observations $(x_1^{(s)},x_2^{(s)},\dots ,x_n^{(s)})$ 
for $s=1,2,\dots, 10$ and $n=7$. Each simulation is the _data_ of 7 dice throws.
Put them into a numpy array

In [ ]:
import numpy as np
import pandas as pd

n = 7
S = 10

x = np.random.randint(size=(S, n), low=1, high=7)
print("shape of x:", x.shape)
print("unique values in x:", np.unique(x))

In `x`, one set of observations is one row. Get one row by indexing into `x`. First index is for the row:

In [ ]:
x

In [ ]:
print("one row:", x[0])    
print("one value:", x[0, 4])

In [ ]:
print("one row:", x[-1])
print("one value:", x[-1, -1])

Numpy offers many convenient and flexible ways to slice and get a subset of numpy arrays (Chapter 2 in Vanderplas). In particular, [_views_ of arrays](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html#Subarrays-as-no-copy-views) are useful. This is in contrast to R that always creates copies (even in function calls).

Numpy makes array operations simple. For example, we can't use add a value to the list:

In [ ]:
## [1,2,3]+3  ## causes error

In addition to common arithmetic functions (called ufuncs) other useful functions are:

In [ ]:
np.multiply.outer(x[0],x[1]) ## outer product of first two rows

In [ ]:
np.add.outer(x[:,0],x[:,1]) ## outer sum of first two columns

Note that sample mean of one set of random observations can be computed by:

In [ ]:
print("object method:", x[0].mean(), "function:", np.mean(x[0]))

Internally, they are equivalent.

In [ ]:
x0 = x[0]
## x0.mean??
## np.mean??

Mean of one row of `x` is sample mean of one simulation: i.e., mean of 50 dice throws. 
How do you compute mean of each simulation? Numpy `axis` refers to the direction of the array. 

Giving argument `axis=0`, the mean function computes column averages.

In [ ]:
temp = x.mean(axis=0)
print("x.mean(axis=0)   :", temp)
print("temp.mean(0)[3]  :", temp[3])
print("temp[:,3].mean() :", x[:,3].mean())

Giving argument `axis=1`, the mean function computes row averages.

In [ ]:
xmean = x.mean(axis=1)
print("xmean:", xmean)

Note that `xmean` contains many observations of the sample mean $\hat\mu_n$. Plot the histogram of `xmean`:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # set plot style
from IPython.core.pylabtools import figsize
figsize(11, 9)
plt.hist(xmean)
plt.xlabel('sample mean')
plt.ylabel('count')
plt.show()

## Effect of Sample Size $n$

Chebyshev's inequality says that 
$$ \Pr(|\hat\mu_n - \mu| < \epsilon) > 1 - \frac{\text{Var}(\hat\mu_n)}{\epsilon^2} 
= 1- \frac{\sigma^2}{n\epsilon^2} $$

Recall that $\sigma^2=2.917$ is the variance of outcomes of dice represented as a random variable. 
Fixing the error to $\epsilon = 1$ makes the RHS a function of $n$.

In order to empirically verify Chebyshev's inequality, we will simulate the dice with a function.

In [ ]:
## throw dice
def throw_dice(n=1):
    
    from numpy.random import randint
    
    return(randint(size=n, low=1, high=7))

## throw dice n times, compute mean
def muhat_n(n=1):
    
    return(throw_dice(n).mean())

Above functions return numpy arrays. We will compile results below in a bare list. Then create Pandas data frame from it (more on Pandas later). Some automatic type conversion is possible, but more strict than R.

In [ ]:
n_choices = np.arange(20, 10001, 10)

results = []
for n in n_choices:
    results += [muhat_n(n)] ## appends to list

df = pd.DataFrame({
    'n_throws': n_choices,
    'muhat_n' : results,
})
df[:10]

In [ ]:
figsize(11, 9)
plt.plot(df['n_throws'], df['muhat_n']);
plt.xlabel("Number of dice throws (n)");
plt.ylabel("Average of n dice throws");

We can build on the result by showing where 95% bound according to Chebyshev's inequality.

If we choose $\epsilon = 1$ and $n$ is fixed at 50. Then, plugging in $\sigma^2$, we get:
$$ 
1- \frac{2.917}{n\cdot \epsilon^2} = 0.95\\
\epsilon = \sqrt{\frac{2.917}{0.05\cdot n}}$$

Following computes error threshold $epsilon$ guaranteed by Chebyshev's threshold.

In [ ]:
epsilon_n = np.sqrt(2.917/(0.05*n_choices))
df['epsilon'] = epsilon_n

Plot the new plot
$$ \Pr(|\hat\mu_n - \mu| < \epsilon) > 0.95\\
\Pr(-\epsilon_n < \hat\mu_n - \mu < \epsilon_n)  > 0.95 $$

In [ ]:
figsize(11, 9)
plt.plot((df['n_throws']),  df['muhat_n']-3.5);    ## subtract true mean 3.5
plt.plot((df['n_throws']),  df['epsilon'], '--r');
plt.plot((df['n_throws']), -df['epsilon'], '--r');
plt.xlabel("Number of dice throws (n)");
plt.ylabel("Average of n dice throws");

In [ ]:
## repeat computing mean of n throws s times
def repeat_muhat_n(n=1, s=100):
    
    out = [muhat_n(n) for one in range(s)]
    
    return({ 'min': min(out), 'max': max(out), 'mean': sum(out)/s})

In [ ]:
repeat_muhat_n(10)

In [ ]:
results = []
for n in n_choices:
    results += [repeat_muhat_n(n)] ## appends to list

df_new = pd.DataFrame(results)
df_new['n_throws'] = n_choices
df_new['epsilon'] = epsilon_n
df_new[:10]

In [ ]:
figsize(11, 9)
plt.plot((df_new['n_throws']),  df_new['mean']-3.5, '-g');    ## subtract true mean 3.5
plt.plot((df_new['n_throws']),  df_new['min'] -3.5, ':g');    ## subtract true mean 3.5
plt.plot((df_new['n_throws']),  df_new['max'] -3.5, ':g');    ## subtract true mean 3.5
plt.plot((df_new['n_throws']),  df_new['epsilon'], '--r');
plt.plot((df_new['n_throws']), -df_new['epsilon'], '--r');
plt.xlabel("Number of dice throws (n)");
plt.ylabel("Average of n dice throws");

## Repeat for 90% bound

Plot $\epsilon$ bounds corresponding to 80% and 90% probability. Interpret these bounds.

In [ ]:
epsilon_n = np.sqrt(2.917/(0.1*n_choices))
df_new['epsilon_10'] = epsilon_n

epsilon_n = np.sqrt(2.917/(0.2*n_choices))
df_new['epsilon_20'] = epsilon_n

figsize(11, 9)
plt.plot((df_new['n_throws']),  df_new['mean']-3.5, '-g')    ## subtract true mean 3.5
plt.plot((df_new['n_throws']),  df_new['min'] -3.5, ':g')    ## subtract true mean 3.5
plt.plot((df_new['n_throws']),  df_new['max'] -3.5, ':g')    ## subtract true mean 3.5
plt.plot((df_new['n_throws']),  df_new['epsilon'], '--r')
plt.plot((df_new['n_throws']), -df_new['epsilon'], '--r')
plt.plot((df_new['n_throws']),  df_new['epsilon_10'], '--b')
plt.plot((df_new['n_throws']), -df_new['epsilon_10'], '--b')
plt.plot((df_new['n_throws']),  df_new['epsilon_20'], '--k')
plt.plot((df_new['n_throws']), -df_new['epsilon_20'], '--k')
plt.xlim(0, 3000)
plt.xlabel("Number of dice throws (n)")
plt.ylabel("Average of n dice throws")
plt.show()

## Sampling from distribution vs data

In simulation, we "sample" from the distribution: i.e., observe more measurements from the true underlying process. In the dice example, sampling from the distribution (true underlying process) is equivalent to the ability to execute the function `throw_dice()`. However, in real-world scenarios, we may not have that luxury: e.g., only be able to record some real dice 1000 times.

In this section, we simulate such situation. There is some loaded dice with pmf as follows:

|Outcome: $x$|1     |2     |3     |4     |5     |6     |
|---------   |---   |---   |---   |---   |---   |---   |
|$\Pr(X=x)$  |1/12  |2/12  |2/12  |2/12  |2/12  |3/12  |

Function for throwing such dice can be written as follows:

In [ ]:
def throw_loaded_dice(n=1):
    from numpy.random import choice
    
    d = [1, 2,2, 3,3, 4,4, 5,5, 6,6,6]
    return(choice(d, n, replace=True))

What `d` represents is if one were to throw dice 12 times, one would get 1 exactly once, 2 exactly twice, 3 exactly twice, $\dots$, 6 exactly three times. In terms of proportion (out of 12 times), you get exactly $\Pr(X=x)$.

Now we 'throw' the dice 10,000 times:

In [ ]:
data = throw_loaded_dice(100000)

In real-world situations, we may not have access to the real dice; however, we are given record of 100000 dice throws.

In this case, we can _resample with replacement_ from the data. The data is our _empirical dice_ since we have 10,000 throws, each adding to count of one of the numbers 1 through 6.

We can use the following function to sample from our _data_ instead:

In [ ]:
def choose_from_data(n=1, data_in=None):
    from numpy.random import choice
    
    return(choice(data_in, n, replace=True))

Let's sample from `data` 1000 times:

In [ ]:
pseudo_data = choose_from_data(1000, data)

We can compare the distribution of the _true_ pmf, _empirical_ pmf, and _resampled_ pmf

In [ ]:
outcome, data_counts = np.unique(data, return_counts=True)
outcome, resample_counts = np.unique(pseudo_data, return_counts=True)

pmfs = pd.DataFrame({
    'outcome': np.arange(1, 7),
    'true_pmf': np.array([1, 2, 2, 2, 2, 3])/12,
    'empirical_pmf': data_counts/sum(data_counts),
    'resample_pmf': resample_counts/sum(resample_counts),
}, columns=['outcome','true_pmf','empirical_pmf','resample_pmf'])

pmfs

Above experiment shows that empirical pmf and resample pmf are similar to the truth. This is the basis for using resampled data (such as bootstrap) for estimating sampling distributions, etc. 